In [1]:
!python -V

Python 3.9.12


In [2]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Lasso

In [3]:
pwd

'/home/ubuntu/notebooks'

In [4]:
!pip install pyarrow

In [6]:
Jan_df = pd.read_parquet('./data/fhv_tripdata_2021-01.parquet')
len(Jan_df)

1154112

In [7]:
Jan_df.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009
2,B00013,2021-01-01 00:01:00,2021-01-01 01:51:00,NaN,NaN,None,B00013
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037


In [8]:
#duration = drop off time -pick up time
Jan_df['duration'] = Jan_df.dropOff_datetime - Jan_df.pickup_datetime
Jan_df.duration = Jan_df.duration.apply(lambda td: td.total_seconds()/60)

In [9]:
Jan_df.duration.mean()

19.1672240937939

Data Preparation

In [10]:
#check duration distribution
Jan_df.duration.describe()

count    1.154112e+06
mean     1.916722e+01
std      3.986922e+02
min      1.666667e-02
25%      7.766667e+00
50%      1.340000e+01
75%      2.228333e+01
max      4.233710e+05
Name: duration, dtype: float64

In [11]:
Jan_df.duration.describe(percentiles = [0.95, 0.98,0.99])

count    1.154112e+06
mean     1.916722e+01
std      3.986922e+02
min      1.666667e-02
50%      1.340000e+01
95%      4.725000e+01
98%      6.613333e+01
99%      9.030000e+01
max      4.233710e+05
Name: duration, dtype: float64

In [12]:
data = Jan_df[(Jan_df.duration>=1)&(Jan_df.duration <= 60)]
len(data)

1109826

In [13]:
len( Jan_df)

1154112

In [14]:
1154112-1109826

44286

In [15]:
data.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
0,B00009,2021-01-01 00:27:00,2021-01-01 00:44:00,NaN,NaN,None,B00009,17.000000
1,B00009,2021-01-01 00:50:00,2021-01-01 01:07:00,NaN,NaN,None,B00009,17.000000
3,B00037,2021-01-01 00:13:09,2021-01-01 00:21:26,NaN,72.0,None,B00037,8.283333
4,B00037,2021-01-01 00:38:31,2021-01-01 00:53:44,NaN,61.0,None,B00037,15.216667
5,B00037,2021-01-01 00:59:02,2021-01-01 01:08:05,NaN,71.0,None,B00037,9.050000


In [16]:
features = ['PUlocationID','DOlocationID']

In [17]:
data[features] 

,PUlocationID,DOlocationID
0,NaN,NaN
1,NaN,NaN
3,NaN,72.0
4,NaN,61.0
5,NaN,71.0
...,...,...
1154107,7.0,7.0
1154108,44.0,91.0
1154109,171.0,171.0
1154110,15.0,15.0


In [18]:
data[features].isna().sum()

PUlocationID    927008
DOlocationID    147907
dtype: int64

In [19]:
new_data = data[features]
new_data.head()

,PUlocationID,DOlocationID
0,NaN,NaN
1,NaN,NaN
3,NaN,72.0
4,NaN,61.0
5,NaN,71.0


In [20]:
new_data.fillna(-1, inplace = True)

/tmp/ipykernel_2199/3586317465.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_data.fillna(-1, inplace = True)


In [21]:
new_data.head()

,PUlocationID,DOlocationID
0,-1.0,-1.0
1,-1.0,-1.0
3,-1.0,72.0
4,-1.0,61.0
5,-1.0,71.0


In [22]:
len(new_data['PUlocationID'][new_data['PUlocationID']== -1.0])/len(new_data['PUlocationID'])

0.8352732770722617

One hot encoding

In [23]:
#Turn the dataframe into a list of dictionaries
new_data =new_data.astype(str)
train_dicts = new_data.to_dict(orient = 'records')

In [24]:
#Fit a dictionary vectorizer
dv = DictVectorizer()
#Get a feature matrix from it
X_train = dv.fit_transform(train_dicts)
print('The dimensionality of the matrix(no of columns) is', X_train.shape[1])

The dimensionality of the matrix(no of columns) is 525


In [25]:
target = 'duration'
y_train = data[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

print('The RMSE of the model ',(mean_squared_error(y_train, y_pred, squared=False)))

The RMSE of the model  10.528519107206316


In [30]:
def read_dataframe(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds()/60)

    df = df[(df.duration>=1)&(df.duration <= 60)]
    return df

In [31]:
df_val = read_dataframe('./data/fhv_tripdata_2021-02.parquet')

In [29]:
df_val

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number,duration
1,B00021,2021-02-01 00:55:40,2021-02-01 01:06:20,173.0,82.0,None,B00021,10.666667
2,B00021,2021-02-01 00:14:03,2021-02-01 00:28:37,173.0,56.0,None,B00021,14.566667
3,B00021,2021-02-01 00:27:48,2021-02-01 00:35:45,82.0,129.0,None,B00021,7.950000
4,B00037,2021-02-01 00:12:50,2021-02-01 00:26:38,NaN,225.0,None,B00037,13.800000
5,B00037,2021-02-01 00:00:37,2021-02-01 00:09:35,NaN,61.0,None,B00037,8.966667
...,...,...,...,...,...,...,...,...
1037687,B03282,2021-02-28 23:01:16,2021-02-28 23:14:48,NaN,31.0,None,B01717,13.533333
1037688,B03282,2021-02-28 23:36:10,2021-02-28 23:47:38,NaN,169.0,None,B01717,11.466667
1037689,B03285,2021-02-28 23:18:36,2021-02-28 23:43:59,28.0,171.0,None,B03285,25.383333
1037690,B03285,2021-02-28 23:26:34,2021-02-28 23:44:37,16.0,252.0,None,B03285,18.050000


In [32]:
features = ['PUlocationID','DOlocationID']
val_data = df_val[features]
val_data.fillna(-1, inplace = True)

val_data =val_data.astype(str)
val_dicts = val_data.to_dict(orient = 'records')
#Get a feature matrix from it
X_val = dv.transform(val_dicts)
    

/tmp/ipykernel_2199/380310843.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  val_data.fillna(-1, inplace = True)


In [33]:
target = 'duration'
y_val = df_val[target].values

#lr = LinearRegression()
#lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

print('The RMSE of the model ',(mean_squared_error(y_val, y_pred, squared=False)))

The RMSE of the model  11.014283149347039
